## Preamble

In [ ]:
# Required packages
import numpy as np
import cv2 
from scipy.signal import savgol_filter
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from math import *
import tifffile
import warnings
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from scipy.optimize import curve_fit
import string
from pylab import *
warnings.filterwarnings("ignore")

In [ ]:
# Plot formatting 
matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams['font.family']='STIXGeneral'
plt.rcParams['legend.fontsize']=15
plt.rcParams.update({'font.size':15}) 
plt.rcParams['axes.axisbelow']=True

## Finding Completeness Magnitude and best fit a (exponent stability)

from: https://github.com/sachalapins/bvalues 

In [ ]:
###### Frequency-magnitude distribution function (Mignan & Woessner article) ######
# mag: catalogue of magnitudes
# mbin: magnitude bin size
def fmd(mag, mbin):
    minmag = math.floor(min(mag/mbin)) * mbin # Lowest magnitude bin
    maxmag = math.ceil(max(mag/mbin)) * mbin # Highest magnitude bin
    mi = np.arange(minmag, maxmag + mbin, mbin) # Sequence of magnitude bins
    nbm = len(mi) # No. of magnitude bins
    cumnbmag = np.zeros(nbm) # Pre-allocate array for cumulative no. of events in mag bin and higher

    # Get cumulative no. of events in mag bin and higher
    for i in range(nbm):
        cumnbmag[i] = np.where(mag > mi[i] - mbin/2)[0].shape[0]
        
    # Get no. of events in each mag bin:
    nbmag = abs(np.diff(np.append(cumnbmag, 0)))
    
    return mi, nbmag, cumnbmag # Return magnitude bins, no. of events in bin, and cumulative no. of events

In [ ]:
###### Function to get completeness magnitude with MAXC method ######
def get_maxc(mag, mbin):
    this_fmd = fmd(mag, mbin) # FMD
    maxc = this_fmd[0][np.argmax(this_fmd[1])] # Mag bin with highest no. of events
    return round(maxc, 1)

In [ ]:
###### Function to estimate b-value using maximum likelihood method (Aki, 1965), original Aki error estimate (Aki, 1965), and Shi & Bolt (1982) improved uncertainty estimate: 
# mag: catalogue of magnitudes
# mbin: magnitude bin size
# mc: completeness magnitude
def b_est(mag, mbin, mc):
    
    mag_above_mc = mag[np.where(mag > round(mc,1)-mbin/2)[0]].values # Magnitudes for events larger than cut-off magnitude mc
    n = mag_above_mc.shape[0] # No of. events larger than cut-off magnitude mc
    if n < 2:
        a = np.nan
        b = np.nan
        aki_unc = np.nan
        shibolt_unc = np.nan
    else:
        mbar = np.mean(mag_above_mc) # Mean magnitude for events larger than cut-off magnitude mc
        b = math.log10(math.exp(1)) / (mbar - (mc - mbin/2)) # b-value from Eq 3
        a = math.log10(n) + b * mc # 'a-value' for Eq 2
        aki_unc = b / math.sqrt(n) # Uncertainty estimate from Eq 4
        shibolt_unc = 2.3 * b**2 * math.sqrt(sum((mag_above_mc - mbar)**2) / (n * (n-1))) # Uncertainty estimate from Eq 5
        

    return a, b, aki_unc, shibolt_unc # Return b-value and estimates of uncertainty

In [ ]:
###### Method of b-Value Stability ######
def get_mbs(mag, mbin, dM = 0.4, min_mc = -3):

    this_fmd = fmd(mag, mbin) # FMD
    this_maxc = get_maxc(mag, mbin) # Needed further down

    # Zeros to accommodate synthetic GR distributions for each magnitude bin
    a = np.zeros(this_fmd[0].shape[0]) # Pre-allocate array to accommodate a values from Eq 2
    b = np.zeros(this_fmd[0].shape[0]) # Pre-allocate array to accommodate b values from Eq 2 & 3
    b_avg = np.zeros(this_fmd[0].shape[0]) # Pre-allocate array to accommodate average b values from Eq 7
    shibolt_unc = np.zeros(this_fmd[0].shape[0]) # Pre-allocate array to accommodate uncertainty values from Eq 5

    # Loop through each magnitude bin, using it as cut-off magnitude
    for i in range(this_fmd[0].shape[0]):
        mi = round(this_fmd[0][i], 1) # Cut-off magnitude
        
        # make the correction for the fdm: ### new ###
        #beta = 2.3026*b[i] # in natural log
        #alpha = 2.3026*a[i] # in natural log
        #exp_a_0 = (1-math.exp(-beta*mbin)) * (r*math.exp(beta*(Mmin + 0.5*mbin))) / (1 - math.exp(-beta(Mmax - Mmin)))
        #fdm_corrected = exp_a_0 / math.exp(beta*mi)
    
        if this_fmd[2][i] > 1:
      
            a[i], b[i], tmp1, shibolt_unc[i] = b_est(mag, mbin, mi) # a and b-values for this cut-off magnitude
        else:
            a[i] = np.nan
            b[i] = np.nan
            shibolt_unc[i] = np.nan

    # Loop through again, calculating rolling average b-value over following dM magnitude units
    no_bins = round(dM/mbin)
    check_bval_stability = []
    for i in range(this_fmd[0].shape[0]):
        if i >= this_fmd[0].shape[0] - (no_bins + 1):
            b_avg[i] = np.nan
            next
        if any(np.isnan(b[i:(i+no_bins+1)])):
            b_avg[i] = np.nan
            check_bval_stability.append(False)
        else:
            b_avg[i] = np.mean(b[i:(i+no_bins+1)])
            check_bval_stability.append(abs(b_avg[i] - b[i]) <= shibolt_unc[i])

    if any(check_bval_stability):
        bval_stable_points = this_fmd[0][np.array(check_bval_stability)]
        mc = round(min(bval_stable_points[np.where(bval_stable_points > min_mc)[0]]), 1) # Completeness mag is first mag bin that satisfies Eq 7
    else:
        mc = this_maxc # If no stability point, use MAXC

    return mc, this_fmd[0], b, b_avg, shibolt_unc

## Function to be optimised with L2-norm in grid-search algorithm.

#### To find best fit (1) exponent of the power-law (correction of AKI's method) AND (2) bin-width of the crack length catalogue (in AKI's method)

In [ ]:
def L2func(m, Lambda, dm):
    
    ''' Function to minimise equation 17 of Geffers et al 2023 (https://doi.org/10.1093/gji/ggac436), 
        to find optimum parameters Lambda and dm:
        Lambda = exponent of the power-law, correction of AKI's method
        dm = bin-width of the crack length catalogue, used in AKI's method
        
        Returns the completeness magnitude (mc) calculated by the MBS method 
        and the output of eq. 17 of Geffers et al 2023.
    '''
    # maximum crack length
    omega = np.max(m) 
    
    # window length where MBS calculates b-vlaue (assumed to be 5xbin_width)
    M_range = 5*dm
    
    # apply MBS method 
    mc, bins, b_AKI, b_avg, shibolt_unc = get_mbs(m, dm, M_range, min_mc = -5)
    
    # calculate b-value from MBS mc output
    cat_a, cat_b, cat_aki_unc, cat_shibolt_unc = b_est(mag = m, mbin = dm, mc = mc)
    
    # convert b-value to Lambda (i.e. from base 10 to base e)
    Lambda_AKI = cat_b/np.log10(np.exp(1))
    
    # equation 17 in Geffers et al, 2023 to be minimised
    opt = (1/Lambda)*(1-(np.exp(Lambda*dm)-1)/(np.exp(Lambda*(omega - mc))-1)) - (1/Lambda_AKI)
    
    
    return opt**2, mc

## Grid-search algorithm:

### 1. Import Data

In [ ]:
path = 'Residual_Thresholding_Fiji_output/whole_image/35-nt_s2.csv' # path to csv file with crack length data
df_raw = pd.read_csv(path, delimiter=',') 
x = np.log(df_raw['Major']*0.00791*1000) #convert to microns

### 2. Grid search to minimise EQ 17 in Geffers et al. (2017)

In [ ]:
#Lambda values
Lambda2search = np.arange(1.2/np.log10(np.exp(1)), 2.5/np.log10(np.exp(1)) + 0.1, 0.1)
#dm values
dm2search = np.arange(0.02, 0.25 + 0.005, 0.005)
#2D space of Lambda and dm
L2 = np.ones([len(dm2search),len(Lambda2search)]) #for output of opt
mcL2 = np.ones([len(dm2search),len(Lambda2search)]) #for mc

for dm_i, Dm in enumerate(dm2search):
    for L_i, L in enumerate(Lambda2search):
        L2_i, mcL2_i = L2func(x, L, Dm) #call Function and obtain opt output and mc
        L2[dm_i,L_i] = L2_i #assign opt output to grid-search grid
        mcL2[dm_i,L_i] = mcL2_i #assign mc output to mcL2 grid

### 3. Find the minimum in the grid and get the corresponding parameters

In [ ]:
# minimum in L2 eq
minL2 = [np.where(L2==np.min(L2))[0][0], np.where(L2==np.min(L2))[1][0]]
# dm that corresponds to that min in L2
optdm = dm2search[minL2[0]]#dm2search[np.where(L2==np.min(L2))[0][0]]#dm2search[20]#
# Lambda that corresponds to that min in L2
optLambda = Lambda2search[minL2[1]]
# mc that corresponds to that min in L2
optMc = mcL2[minL2[0], minL2[1]] 
# b-value in base 10
opt_b = np.log10(np.exp(1))*optLambda
# mc in base 10
log10Lc = np.log10(np.exp(1))*optMc
optdm,  optLambda 

### 4. Plot results

In [ ]:
###FIGURE CONF
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(19,7))
bottom, top = 0, 0.8
left, right = 0, 0.9
fig.subplots_adjust(top = top, bottom = bottom, left = left, right = right, hspace = 0.3, wspace = 0.3)


###########################################################
#          LEFT         #
# gridsearch output
im = ax[0].imshow(L2, vmin=0, vmax=0.001, origin="lower", 
           extent=(Lambda2search[0]*np.log10(np.exp(1)),Lambda2search[-1]*np.log10(np.exp(1)),
                   dm2search[0]*np.log10(np.exp(1)),dm2search[-1]*np.log10(np.exp(1))), 
           aspect="auto")
# absolute minimum 
ax[0].scatter(Lambda2search[np.where(L2==np.min(L2))[1][0]]*np.log10(np.exp(1)),
             dm2search[np.where(L2==np.min(L2))[0][0]]*np.log10(np.exp(1)),c="red")
#ax[0].scatter(Lambda2search[12]*np.log10(np.exp(1)),dm2search[20]*np.log10(np.exp(1)),c="red")
# colorbar
fmt = matplotlib.ticker.ScalarFormatter(useMathText=True)
fmt.set_powerlimits((0, 0))
fig.colorbar(im, label="Objective [L2]", ax=ax[0],format=fmt)
ax[0].set_xlabel(r"a") ; ax[0].set_ylabel(r"$dm$")



###########################################################
#          RIGHT         #
## calling functions

cat_mi, cat_nbmag, cat_cumnbmag = fmd(mag = x, mbin = optdm)
cat_mbs = get_mbs(mag = x, mbin = optdm, dM = optdm*5) # MBS completeness magnitude
cat_mc = cat_mbs[0] # GFT completeness magnitude
cat_a, cat_b, cat_aki_unc, cat_shibolt_unc = b_est(mag = x, mbin = optdm, mc = cat_mc)

## plotting results
ax[1].scatter([], [], c='red', label = r'Min$_{glob}$(L2)')
ax[1].axvline(log10Lc , color = 'black' , ls = '--' , label = r'MBS $l_c$')
ax[1].scatter(cat_mi/np.log(10), cat_cumnbmag, marker = 's', color = 'black', alpha = 0.6, edgecolor = 'black'
          , label = 'Cumulative no. of Cracks')
ax[1].scatter(cat_mi/np.log(10), cat_nbmag, marker = 'o', color = 'black', alpha = 0.6, edgecolor = 'black'       , label = 'No. of Cracks')
ax[1].plot(cat_mi/np.log(10), (10**(cat_a-0.65 - (opt_b * cat_mi))), color = 'dodgerblue', label = r'$\log_{n}{(F)}= c - al$')
ax[1].set_yscale('log')
ax[1].grid()
ax[1].set_xlabel(r'log$_{10}\left( l \right)$ ($\mu$m)')
ax[1].set_ylabel('Frequency')
ax[1].text(x=log10Lc+0.04, y=3e4, s=r'$l_c$ = ' + str(round(10**(log10Lc), 1))+r'$\mu$m')
ax[1].text(x=log10Lc+0.04, y=1e4, s=r'$a$ = ' + str(round(opt_b, 1)) + " $\pm$ " + str(round(cat_shibolt_unc, 1)))
ax[1].set_xlim(1, 3)
ax[1].set_ylim(0.9, 10**5)
ax[1].legend(loc="lower left", bbox_to_anchor = (-1.25, -0.365), ncol=5, columnspacing = 0.7, handletextpad = 0.05)
#### Label Subplots
for n, a in enumerate(ax.flat):
    a.text(-0.15, 0.98, string.ascii_lowercase[n]+")", transform = a.transAxes, size=22, weight = 'bold')
print(opt_b, cat_b)
fig.savefig('PAPER FIGURES/GridSearch_Sk35.png', bbox_inches='tight', dpi= 200)
plt.show()

## Processing all images of both methods

The output of the following cells is in the csv files: Evolution_SK_all.csv, Evolution_RT_all.csv. 


#### 1. Residual Thresholding

In [ ]:
'''

bval_GT = []
Mc_GT = []

Lambda2search = np.arange(1.2/np.log10(np.exp(1)), 2.5/np.log10(np.exp(1)) + 0.1, 0.1)
#dm values
dm2search = np.arange(0.02, 0.25 + 0.005, 0.005)

for ff in range(0,36,2):
    #DATA
    df_raw = pd.read_csv('Residual_Thresholding_Fiji_output/whole_image/'+str(ff)+'-nt_s2.csv', delimiter=',') #import data
    x = np.log(df_raw['Major']*0.00791*1000) #convert to microns
    
    #OPTIMISATION OF LAMBDA AND DM
    #2D space of Lambda and dm
    L2 = np.ones([len(dm2search),len(Lambda2search)]) #for output of opt
    mcL2 = np.ones([len(dm2search),len(Lambda2search)]) #for mc

    for dm_i, Dm in enumerate(dm2search):
        for L_i, L in enumerate(Lambda2search):
            L2_i, mcL2_i = L2func(x, L, Dm) #call Function and obtain opt output and mc
            L2[dm_i,L_i] = L2_i #assign opt output to grid-search grid
            mcL2[dm_i,L_i] = mcL2_i #assign mc output to mcL2 grid

    minL2 = [np.where(L2==np.min(L2))[0][0], np.where(L2==np.min(L2))[1][0]]
    # dm that corresponds to that min in L2
    optdm = dm2search[minL2[0]]#dm2search[np.where(L2==np.min(L2))[0][0]]#dm2search[20]#
    # Lambda that corresponds to that min in L2
    optLambda = Lambda2search[minL2[1]]#
    # mc that corresponds to that min in L2
    optMc = mcL2[minL2[0], minL2[1]] 
    # b-value in base 10
    opt_b = np.log10(np.exp(1))*optLambda
    # mc in base 10
    log10Lc = np.log10(np.exp(1))*optMc
    print(ff)
    #APPEND VALUES 
    bval_GT.append(opt_b)
    Mc_GT.append(log10Lc)

'''

In [ ]:
'''

a = np.array(bval_GT)
b = np.array(Mc_GT)
c = np.arange(0,36,2)

df = pd.DataFrame({"Image":c, "b_val" : a, "Mc" : b})
df.to_csv("Evolution_GT_all.csv", index=False)

'''

#### 2. Skeletonisation

In [ ]:
'''

bval_SK = []
Mc_SK = []

Lambda2search = Lambda2search = np.arange(2.5/np.log10(np.exp(1)), 5.5/np.log10(np.exp(1)) + 0.3, 0.3)
#dm values
dm2search = np.arange(0.02, 0.25 + 0.005, 0.005)

for ff in range(0,36,2):
    #DATA
    df_raw = pd.read_csv('Griff_new_all/FIJI_out_s1/whole/'+str(ff)+'-s1_lukeout.csv', delimiter=',') #import data

    x = np.log(df_raw['Major']*0.00791*1000) #convert to microns
    
    #OPTIMISATION OF LAMBDA AND DM
    #2D space of Lambda and dm
    L2 = np.ones([len(dm2search),len(Lambda2search)]) #for output of opt
    mcL2 = np.ones([len(dm2search),len(Lambda2search)]) #for mc

    for dm_i, Dm in enumerate(dm2search):
        for L_i, L in enumerate(Lambda2search):
            L2_i, mcL2_i = L2func(x, L, Dm) #call Function and obtain opt output and mc
            L2[dm_i,L_i] = L2_i #assign opt output to grid-search grid
            mcL2[dm_i,L_i] = mcL2_i #assign mc output to mcL2 grid

    minL2 = [np.where(L2==np.min(L2))[0][0], np.where(L2==np.min(L2))[1][0]]
    # dm that corresponds to that min in L2
    optdm = dm2search[minL2[0]]#dm2search[np.where(L2==np.min(L2))[0][0]]#dm2search[20]#
    # Lambda that corresponds to that min in L2
    optLambda = Lambda2search[minL2[1]]#
    # mc that corresponds to that min in L2
    optMc = mcL2[minL2[0], minL2[1]] 
    # b-value in base 10
    opt_b = np.log10(np.exp(1))*optLambda
    # mc in base 10
    log10Lc = np.log10(np.exp(1))*optMc
    print(ff)
    #APPEND VALUES 
    bval_SK.append(opt_b)
    Mc_SK.append(log10Lc)

'''

In [ ]:
'''

a1 = np.array(bval_SK)
b1 = np.array(Mc_SK)
c1 = np.arange(0,36,2)

df1 = pd.DataFrame({"Image":c1, "b_val" : a1, "Mc" : b1})
df1.to_csv("Evolution_SK_all.csv", index=False)

'''

Plotting the output of the previous cell

In [ ]:
E_SK = pd.read_csv("Evolution_SK_all.csv")
E_GT = pd.read_csv("Evolution_GT_all.csv")

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
bottom, top = 0, 0.8
left, right = 0, 0.9
fig.subplots_adjust(top = top, bottom = bottom, left = left, right = right, hspace = 0.3, wspace = 0.2)

#evolution a value
ax[0].plot(E_SK['Image'],E_SK['b_val'], c ='darkorange', label='SK', lw=3)
ax[0].plot(E_GT['Image'],E_GT['b_val'], c ='dodgerblue', label='RT', lw=3)

#evolution completeness length
ax[1].plot(E_SK['Image'],10**E_SK['Mc'], c ='darkorange', lw=3)
ax[1].plot(E_GT['Image'],10**E_GT['Mc'], c ='dodgerblue', lw=3)

#peak stress
ax[0].axvline(x = 18, label = 'Peak Stress', c = 'k', ls = '--', lw=2)
ax[1].axvline(x = 18, c= 'k', ls = '--', lw=2)

#average A values
ax[0].hlines(y = E_SK['b_val'].iloc[0:10].mean(), xmin=0, xmax=18,colors ='darkorange', lw=2, ls='dotted' )
ax[0].hlines(y = E_SK['b_val'].iloc[10:].mean(), xmin=18, xmax=34,colors ='darkorange', lw=2, ls='dotted' )
ax[0].hlines(y = E_GT['b_val'].iloc[0:10].mean(), xmin=0, xmax=18,colors ='dodgerblue', lw=2, ls='dotted' )
ax[0].hlines(y = E_GT['b_val'].iloc[10:].mean(), xmin=18, xmax=34,colors ='dodgerblue', lw=2, ls='dotted' , label = 'Mean')
#average Mc values
ax[1].hlines(y = 10**E_SK['Mc'].iloc[0:10].mean(), xmin=0, xmax=18,colors ='darkorange', lw=2, ls='dotted' )
ax[1].hlines(y = 10**E_SK['Mc'].iloc[10:].mean(), xmin=18, xmax=34,colors ='darkorange', lw=2, ls='dotted' )
ax[1].hlines(y = 10**E_GT['Mc'].iloc[0:10].mean(), xmin=0, xmax=18,colors ='dodgerblue', lw=2, ls='dotted' )
ax[1].hlines(y = 10**(E_GT['Mc'].iloc[10:].mean()), xmin=18, xmax=34,colors ='dodgerblue', lw=2, ls='dotted' )

#fill between
#ax[0].fill_between(, y1, y2=0, where=None, interpolate=False, step=None, *, data=None, **kwargs)
ax[0].fill_between(x = E_SK['Image'].iloc[0:10], y1 = E_SK['b_val'].iloc[0:10].mean()-E_SK['b_val'].iloc[0:10].std()
                   ,y2 = E_SK['b_val'].iloc[0:10].mean()+E_SK['b_val'].iloc[0:10].std(),color ='darkorange',
                  alpha = 0.1)
ax[0].fill_between(x = E_SK['Image'].iloc[9:], y1 = E_SK['b_val'].iloc[10:].mean()-E_SK['b_val'].iloc[5:].std()
                   ,y2 = E_SK['b_val'].iloc[10:].mean()+E_SK['b_val'].iloc[10:].std(),color ='darkorange',
                  alpha = 0.1)
ax[0].fill_between(x = E_GT['Image'].iloc[0:10], y1 = E_GT['b_val'].iloc[0:10].mean()-E_GT['b_val'].iloc[0:10].std()
                   ,y2 = E_GT['b_val'].iloc[0:10].mean()+E_GT['b_val'].iloc[0:10].std(),color ='dodgerblue',
                  alpha = 0.1 )
ax[0].fill_between(x = E_GT['Image'].iloc[9:], y1 = E_GT['b_val'].iloc[10:].mean()-E_GT['b_val'].iloc[10:].std()
                   ,y2 = E_GT['b_val'].iloc[10:].mean()+E_GT['b_val'].iloc[10:].std(),color ='dodgerblue',
                  alpha = 0.1 , label = 'Std')



ax[1].fill_between(x = E_SK['Image'].iloc[0:10], y1 = 10**(E_SK['Mc'].iloc[0:10].mean()-E_SK['Mc'].iloc[0:10].std())
                   ,y2 = 10**(E_SK['Mc'].iloc[0:10].mean()+E_SK['Mc'].iloc[0:10].std()),color ='darkorange',
                  alpha = 0.1)
ax[1].fill_between(x = E_SK['Image'].iloc[9:], y1 = 10**(E_SK['Mc'].iloc[10:].mean()-E_SK['Mc'].iloc[10:].std())
                   ,y2 = 10**(E_SK['Mc'].iloc[10:].mean()+E_SK['Mc'].iloc[10:].std()),color ='darkorange',
                  alpha = 0.1)
ax[1].fill_between(x = E_GT['Image'].iloc[0:10], y1 = 10**(E_GT['Mc'].iloc[0:10].mean()-E_GT['Mc'].iloc[0:10].std())
                   ,y2 = 10**(E_GT['Mc'].iloc[0:10].mean()+E_GT['Mc'].iloc[0:10].std()),color ='dodgerblue',
                  alpha = 0.1 )
ax[1].fill_between(x = E_GT['Image'].iloc[9:], y1 = 10**(E_GT['Mc'].iloc[10:].mean()-E_GT['Mc'].iloc[10:].std())
                   ,y2 = 10**(E_GT['Mc'].iloc[10:].mean()+E_GT['Mc'].iloc[10:].std()),color ='dodgerblue',
                  alpha = 0.1 )


#customising
ax[0].set_xlabel('Image Number')
ax[1].set_xlabel('Image Number')
ax[0].set_ylabel(r'$a$')
ax[1].set_ylabel(r'$l_{c}$ ($\mu$m)')
#ax[0].plot([], [], c = 'dodgerblue', alpha = 0.1, label = 'Std')

ax[0].legend(loc="upper left", bbox_to_anchor = (0.08, 1.25),ncol = 5)
ax[0].grid()
ax[1].grid()
for n, a in enumerate(ax):
    a.text(-0.099, 0.96, string.ascii_lowercase[n]+")", transform = a.transAxes, size=22, weight = 'bold')
fig.savefig('PAPER FIGURES/evolution_lc_a.png', bbox_inches='tight', dpi= 200)        